In [1]:
from util import *
from preprocess import *

WORD_IMG_SHAPE = (64, 64 * 4)
LATENT_SIZE = np.prod(WORD_IMG_SHAPE)
BATCH_SIZE = 32

In [2]:
from transformers import VisionEncoderDecoderModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten", num_labels=27)
model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, 

In [7]:
from datasets import load_metric

cer = load_metric("cer")

In [8]:
from tqdm.notebook import tqdm
import torchvision.transforms.functional as VF

print("Running evaluation...")

data_loader = DataLoader(load_dataset('char', equal_shapes=False), shuffle=True, num_workers=0, pin_memory=True)
cwg = CorruptWordGen(data_loader, batch_size=BATCH_SIZE, img_shape=WORD_IMG_SHAPE)

for base_imgs, crpt_imgs, labels in tqdm(cwg):
    # predict using generate
    # pixel_values = batch["pixel_values"].to(device)
    batch_size, channel, height, width = crpt_imgs.shape
    input = torch.zeros((batch_size, 3, height, width), dtype=crpt_imgs.dtype, device=device)
    input[:,:,:,:] = crpt_imgs.clone()
    outputs = model.generate(VF.resize(input, (384, 384)))
    
    print(outputs.shape)
    break

    # decode
    pred_str = processor.batch_decode(outputs, skip_special_tokens=True)
    labels = batch["labels"]
    labels[labels == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels, skip_special_tokens=True)

    # add batch to metric
    cer.add_batch(predictions=pred_str, references=label_str)

final_score = cer.compute()

Running evaluation...


  0%|          | 0/35 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


torch.Size([32, 11])


ValueError: Metric cache file doesn't exist. Please make sure that you call `add` or `add_batch` at least once before calling `compute`.

In [ ]:
print("Character error rate on test set:", final_score)

Character error rate on test set: 0.026648996932397223
